In [1]:
# import required modules
import requests
from bs4 import BeautifulSoup
import pandas as pd
import geopy
# get URL
page = requests.get("https://en.wikipedia.org/wiki/List_of_rugby_union_competitions")
soup = BeautifulSoup(page.content, 'html.parser')

In [2]:
#look for all tournament
tournament_links = []
tournament_table = soup.find_all('table')[0]
tournament_row = tournament_table.find_all('tr')

for tournament in tournament_row:
    tf = tournament.find('td')
    a = tournament.find('a',href=True)
    if a:
        if 'Women' not in a['href']:
            tournament_links.append(a['href'][6:])
tournament_links

['Rugby_World_Cup',
 'Six_Nations_Championship',
 'The_Rugby_Championship',
 'Rugby_Europe_International_Championships',
 'World_Rugby_Pacific_Nations_Cup',
 'World_Rugby_Pacific_Challenge',
 'World_Rugby_Nations_Cup',
 'Americas_Rugby_Championship',
 'Asia_Rugby_Championship']

page = requests.get(f"https://en.wikipedia.org/wiki/2022_Rugby_Championship", allow_redirects=False, timeout=5)
soup = BeautifulSoup(page.content, 'html.parser')
fixtures =  soup.find_all(class_='vevent summary')
results = []
for i in range(len(fixtures)):
    try:
        tables = fixtures[i].find_all('table')
        teams = tables[1].find_all('td', class_='vcard')
        team_a = teams[0].find('a').text
        team_b = teams[1].find('a').text
        info = fixtures[i].find('tr')
        td = info.find('td')
        td.select("br")[0].replace_with("\n")
        date= td.text.split('\n')[0]
        #find the score
        table_result = tables[1]
        result = table_result.find_all('td')[1].text
        #find the venue
        venue = tables[2]
        venue.select("br")[0].replace_with("\n")
        venue = venue.text.split('\n')[0]
        results.append([f'{team_a}_v_{team_b}', date, result, venue])
    except Exception as e:
        print(f"An error occurred: {e} {year} {url}")
        continue


In [109]:
def result_scrapping(year, url):
    try:
        page = requests.get(f"https://en.wikipedia.org/wiki/{year}_{url}", allow_redirects=False, timeout=5)
        results = []
        if page.status_code == 200:
            soup = BeautifulSoup(page.content, 'html.parser')
            fixtures =  soup.find_all(class_='vevent summary')
            
            for i in range(len(fixtures)):
                try:
                    tables = fixtures[i].find_all('table')
                    teams = tables[1].find_all('td', class_='vcard')
                    team_a = teams[0].find('a').text
                    team_b = teams[1].find('a').text
            
                    info = fixtures[i].find('tr')
                    td = info.find('td')
                    td.select("br")[0].replace_with("\n")
                    date= td.text.split('\n')[0]
                    #remove trailing info
                    date_str = date[:date.rindex(f'{year}') + len(f'{year}')]
                    #find the score
                    table_result = tables[1]
                    result = table_result.find_all('td')[1].text
                    #find the venue
                    venue = tables[2]
                    venue.select("br")[0].replace_with("\n")
                    venue = venue.text.split('\n')[0]
                    results.append([f'{team_a}_v_{team_b}', date_str, result, venue])
                except Exception as e:
                    print(f"An error occurred: {e} {year} {url}")
                    continue
    except Exception as e:
        print(f"An error occurred while fetching page: {e}")
    return results

# Obtain match results

In [121]:
scapped_results = {}
years= [year for year in range(2021,2024)]
print()
urls = ['Six_Nations_Championship','Rugby_Championship','Rugby_Europe_International_Championships',
 'World_Rugby_Pacific_Nations_Cup',
 'World_Rugby_Pacific_Challenge',
 'World_Rugby_Nations_Cup',
 'Americas_Rugby_Championship',
 'Asia_Rugby_Championship',
       ]
for url in urls:
    tournament_results = [result_scrapping(years[i], url) for i in range(len(years))] 
    scapped_results[url]=tournament_results


An error occurred: list index out of range 2022 Asia_Rugby_Championship
An error occurred: list index out of range 2022 Asia_Rugby_Championship
An error occurred: substring not found 2022 Asia_Rugby_Championship


In [127]:
#create a csv file with the results,
import re
result_df = pd.DataFrame(columns=['Date', 'home_team','away_team','home_score','away_score', 'Tourn','Venue', 'city', 'country', 'Neut.'])
#access each tournament
dates = []
home_score = []
away_score = []
home_team = []
away_team = []
competition = []
stadium = []
city = []
for entry in scapped_results:
    tournaments = scapped_results[entry]
    #access each year
    for i in range(len(tournaments)):
        #access each games
        for j in range(len(tournaments[i])):
            score = tournaments[i][j][2]
            scores = re.split('–|-', score)
            if scores[0] != 'Postponed[b]':
                dates.append(tournaments[i][j][1])
                
                home_score.append(int(scores[0]))
                away_score.append(int(scores[1]))
                teams = tournaments[i][j][0].split('_v_')
                home_team.append(teams[0])
                away_team.append(teams[1])
                competition.append(entry)
                geo = tournaments[i][j][-1].split(', ')
                stadium.append(geo[0])
                city.append(geo[1])
       

In [128]:
date_parsed = [pd.to_datetime(date,dayfirst=True, format='%d %B %Y') for date in dates]
result_df['Date']= date_parsed
result_df['home_score']= home_score
result_df['away_score']= away_score
result_df['home_team']= home_team
result_df['away_team']= away_team
result_df['Tourn']= competition
result_df['Venue']= stadium
result_df['city'] = city

In [139]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent = "AGexercice", timeout=10)
def find_country(city, geo):
    location = geo.geocode(city, language='en')
    if location:

        country_name = location.raw['display_name'].split(', ')[-1]
        if country_name == 'United Kingdom':
            if location.raw['display_name'].split(', ')[-2] != 'England':
                country_name = location.raw['display_name'].split(', ')[1]
            else:
                country_name = 'England'
        return country_name
    return None

In [140]:
result_df.country = result_df.apply(lambda row : find_country(row['city'], geolocator), axis=1 )

In [130]:
#  Add the neutrtal when necessary
result_df['Neut.'] = result_df.apply(lambda row: row['home_team']!=row['country'], axis=1)

In [131]:
result_df

,Date,home_team,away_team,home_score,away_score,Tourn,Venue,city,country,Neut.
0,2021-02-06,Italy,France,10,50,Six_Nations_Championship,Stadio Olimpico,Rome,Italy,False
1,2021-02-06,England,Scotland,6,11,Six_Nations_Championship,Twickenham Stadium,London,England,False
2,2021-02-07,Wales,Ireland,21,16,Six_Nations_Championship,Millennium Stadium,Cardiff,Wales,False
3,2021-02-13,England,Italy,41,18,Six_Nations_Championship,Twickenham Stadium,London,England,False
4,2021-02-13,Scotland,Wales,24,25,Six_Nations_Championship,Murrayfield Stadium,Edinburgh,Scotland,False
...,...,...,...,...,...,...,...,...,...,...
94,2022-11-22,Nepal,Bangladesh,12,43,Asia_Rugby_Championship,Rabindra Sarobar Stadium,Kolkata,India,True
95,2022-11-25,India,Bangladesh,82,0,Asia_Rugby_Championship,Rabindra Sarobar Stadium,Kolkata,India,False
96,2023-06-03,Malaysia,South Korea,3,27,Asia_Rugby_Championship,Siadinha Hamzah Stadium,Kuala Lumpur,Malaysia,False
97,2023-06-10,Hong Kong,Malaysia,88,9,Asia_Rugby_Championship,Hong Kong Football Club Stadium,Hong Kong,China,True


In [132]:
folder = 'data/results_international/'
result_df.to_excel(folder+'scrap_out.xlsx')